In [1]:
import json
import os
from pathlib import Path
import shutil
import yaml
from ultralytics import YOLO

# coco 형식 데이터 -> yolo 형식으로 변환
def convert_coco_to_yolo(coco_file, output_dir, dataset_type, base_dir):
    with open(coco_file, 'r') as f:
        coco_data = json.load(f)
    
    images_dir = os.path.join(output_dir, dataset_type, 'images')
    labels_dir = os.path.join(output_dir, dataset_type, 'labels')
    os.makedirs(images_dir, exist_ok=True) 
    os.makedirs(labels_dir, exist_ok=True) 
    
    image_annotations = {}
    for ann in coco_data['annotations']:
        image_id = ann['image_id'] 
        if image_id not in image_annotations:
            image_annotations[image_id] = [] 
        image_annotations[image_id].append(ann)
    
    # image 정보 순회하며 yolo 형식으로 변환
    for img in coco_data['images']:
        img_id = img['id'] # image id
        img_name = img['file_name'] # image file_name
        img_width = img['width'] # image width
        img_height = img['height'] # image height
        
        src_image = os.path.join(base_dir, img_name)
        dst_image = os.path.join(images_dir, os.path.basename(img_name))
        shutil.copy(src_image, dst_image)
        
        # YOLO 형식 라벨 파일 생성
        label_file = os.path.join(labels_dir, Path(img_name).stem + '.txt')
        with open(label_file, 'w') as f:
            if img_id in image_annotations:
                for ann in image_annotations[img_id]:
                    category_id = ann['category_id']
                    bbox = ann['bbox']
                    # YOLO 형식의 (x_center, y_center, width, height)로 변환
                    x_center = (bbox[0] + bbox[2] / 2) / img_width
                    y_center = (bbox[1] + bbox[3] / 2) / img_height
                    width = bbox[2] / img_width
                    height = bbox[3] / img_height
                    f.write(f"{category_id} {x_center} {y_center} {width} {height}\n")
    
    print(f"Conversion completed for {dataset_type} dataset")

# YOLO datasets에 맞는 YAML file 생성
def create_dataset_yaml(output_dir, train_dir, val_dir, class_names):
    yaml_content = {
        'path': output_dir, # 데이터셋 생성 dir
        'train': train_dir, # train_dir
        'val': val_dir, # val_dir
        'nc': len(class_names), # class 개수
        'names': class_names # 클래스 이름 목록
    }
    
    yaml_path = os.path.join(output_dir, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f)
    
    return yaml_path

def main():
    base_dir = '/data/ephemeral/home/dataset' # 기본 데이터셋 경로
    folds_dir = os.path.join(base_dir, 'folds') # cv fold 데이터 경로
    yolo_dataset_dir = '/data/ephemeral/home/dataset/RTDETR_dataset'
    
    class_names = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
    
    # 5 fold cv 수행
    for fold in range(1,5):
        fold_dir = os.path.join(yolo_dataset_dir, f'fold_{fold}')
        os.makedirs(fold_dir, exist_ok=True)
        
        # 학습 데이터 coco -> yolo 반환
        train_json = os.path.join(folds_dir, f'train_fold{fold}.json')
        convert_coco_to_yolo(train_json, fold_dir, 'train', base_dir)
        
        # 검증 데이터 coco -> yolo 반환
        val_json = os.path.join(folds_dir, f'val_fold{fold}.json')
        convert_coco_to_yolo(val_json, fold_dir, 'val', base_dir)
        
        # 폴드 데이터에 맞는 yaml 파일 생성
        train_dir = os.path.join(fold_dir, 'train', 'images')
        val_dir = os.path.join(fold_dir, 'val', 'images')
        yaml_path = create_dataset_yaml(fold_dir, train_dir, val_dir, class_names)
        
        print(f"Yolo format dataset for fold {fold} has been created at: {fold_dir}")
        
        # YOLO 모델 학습
        model = YOLO("yolo11x.pt")
        model.train(
            data=yaml_path, 
            epochs=100, 
            project="Yolo11", 
            imgsz=640,  # 입력 이미지 크기를 1024x1024로 설정
            batch=16,
            optimizer='AdamW',  # 옵티마이저를 AdamW로 설정
            lr0=1e-3,
            name=f"Yolo11_epoch100_fold{fold}",
            patience=20, # early stopping 20
            iou=0.5,
            label_smoothing=0.1,
            mosaic=True,  # Mosaic 증강 활성화
            flipud=0.5,  # 상하 반전 (50% 확률)
            fliplr=0.5,  # 좌우 반전 (50% 확률)
            hsv_h=0.015,  # 색조(Hue) 변환 범위
            hsv_s=0.7,  # 채도(Saturation) 변환 범위
            hsv_v=0.4,  # 밝기(Value) 변환 범위
            scale=0.5,  # 크기 조정 범위
            shear=0.5,  # 전단 변환 범위
            perspective=0.5,  # 원근 변환 (기본 0)
            degrees=30.0  # 회전 각도를 ±30도로 설정
            )
        print(f"Training completed for fold {fold}")

if __name__ == "__main__":
    main()

Conversion completed for train dataset
Conversion completed for val dataset
Yolo format dataset for fold 1 has been created at: /data/ephemeral/home/dataset/RTDETR_dataset/fold_1
New https://pypi.org/project/ultralytics/8.3.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.15 🚀 Python-3.10.13 torch-1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolo11x.pt, data=/data/ephemeral/home/dataset/RTDETR_dataset/fold_1/data.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=Yolo11, name=Yolo11_epoch100_fold1, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, 

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: leejh_00 (iron8233-naver). Use `wandb login --relogin` to force relogin


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...
AMP: checks passed ✅


train: Scanning /data/ephemeral/home/dataset/RTDETR_dataset/fold_1/train/labels.cache... 3902 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3902/3902 [00:00<?, ?it/s]

train: WARNING ⚠️ /data/ephemeral/home/dataset/RTDETR_dataset/fold_1/train/images/4041.jpg: 1 duplicate labels removed


...

val: Scanning /data/ephemeral/home/dataset/RTDETR_dataset/fold_1/val/labels.cache... 981 images, 0 backgrounds, 0 corrupt: 100%|██████████| 981/981 [00:00<?, ?it/s]


Plotting labels to Yolo11/Yolo11_epoch100_fold1/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to Yolo11/Yolo11_epoch100_fold1
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      17.8G      2.381      6.198      2.476         37        640:  10%|▉         | 24/244 [00:11<01:46,  2.06it/s]


KeyboardInterrupt: 

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.
